# Decorators
- Functions, classes, and methods can be 'decorated'
- Decorators can be complex - will only show how to decorate functions
- Similar to 'annotations and aspect' programming in java
- Good for 'cross cutting' concerns, like security, metering, billing. 
- Surprising what can be done with decorators

# Callables
- a 'callable' is something that can be 'called' - applied to arguments
    - functions and lambdas are callables
    - objects can also be callables, by defining the ```__call__``` method

In [1]:
import math
import time

class Co:
    
    # args applied to object will call this method
    def __call__(self, x):
        return(math.sin(x))

# make a Co object
c = Co()

# can call object like a function
math.sin(.5), c(.5)
    

(0.479425538604203, 0.479425538604203)

In [2]:
# predicate 

[(obj, callable(obj)) for obj in [5, "asdf", math.sin, object(), c]]

[(5, False),
 ('asdf', False),
 (<function math.sin(x, /)>, True),
 (<object at 0x10d72a790>, False),
 (<__main__.Co at 0x10dfdd6d0>, True)]

# to decorate a function, define a callable class

In [3]:
class timefunc:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *pos, **kw):
        start = time.time()
        # call the "original function"
        # hang on to return value
        val = self.func(*pos, **kw)
        
        # compute and print runtime
        interval = time.time() - start
        #print('execution took {}'.format(interval))
        print(f'execution took {interval}')
        
        # return the function value
        return val
 

In [4]:
# decorators always start with '@'
@timefunc

def run(n, faster=None):
    s = n/2 if faster else n
    time.sleep(s)
    return n * n

# what  happened in cell above?
- a function object was created, specified by the def run statement
- an instance of timefunc was created, and the function object was passed to the init method
- note use of ```*pos and **kw``` to pass thru any possible set of function args
- the name 'run' was set to the instance of timefunc(a callable)

In [5]:
# run is instance of timefunc

run, callable(run)

(<__main__.timefunc at 0x10e050c50>, True)

In [6]:
# invokes call method of timefunc object

run(1)

execution took 1.003420114517212


1

In [7]:
# args passed thru correctly

run(4, faster=True)

execution took 2.005188226699829


16

# more complex example - tracing function execution

In [8]:
# good old recursive factorial, 
# with a print debug statement added

def fact(n):
    print('inside fact({})'.format(n))
    if n == 0:
        return(1)
    else:
        return(n * fact(n-1))

fact(4)


inside fact(4)
inside fact(3)
inside fact(2)
inside fact(1)
inside fact(0)


24

In [9]:
class traceindent:
    def __init__(self, func):
        # func is the original function
        # defined below @traceident line
        self.func = func
        self.level = 0

    # when func is called - this method
    # is called, not the original func
    # grab all args with *pos and **kw
    def __call__(self, *pos, **kw):
        # level count 
        self.level += 1
        indent = ['|'] * self.level
        indent = ''.join(indent)
        if len(pos) == 1:
            printpos = '({})'.format(pos[0])
        print(f'{indent}Entering({self.level}) {self.func.__name__}{printpos}')
        # call the traced function
        val = self.func(*pos, **kw)
        print(f'{indent}Exiting({self.level}) {self.func.__name__}{printpos}=>{val}')              
        self.level -= 1
        return(val)


In [10]:
# removed the print statement from fact
# 'decorate' the fact function with a traceindent

@traceindent
def fact(n):
    if n == 0:
        return(1)
    else:
        return(n * fact(n-1))

In [11]:
fact(2)

|Entering(1) fact(2)
||Entering(2) fact(1)
|||Entering(3) fact(0)
|||Exiting(3) fact(0)=>1
||Exiting(2) fact(1)=>1
|Exiting(1) fact(2)=>2


2

In [12]:
fact(4)

|Entering(1) fact(4)
||Entering(2) fact(3)
|||Entering(3) fact(2)
||||Entering(4) fact(1)
|||||Entering(5) fact(0)
|||||Exiting(5) fact(0)=>1
||||Exiting(4) fact(1)=>1
|||Exiting(3) fact(2)=>2
||Exiting(2) fact(3)=>6
|Exiting(1) fact(4)=>24


24

In [13]:
# easy to use on another function

@traceindent
def rcount(x):
    if isinstance(x, list):
        # x is a list, get the length
        xlen = len(x)
        if xlen == 0:
            return 0
        if xlen == 1:
            return(rcount(x[0]))
        else:
            # use an index access and a slice
            # to subdivide list into head and tail
            return rcount(x[0]) + rcount(x[1:])

    # x is not a list, so just counts as 1
    return(1)

In [14]:
rcount([1,2,[3,4,[5,6,7],8],9])

|Entering(1) rcount([1, 2, [3, 4, [5, 6, 7], 8], 9])
||Entering(2) rcount(1)
||Exiting(2) rcount(1)=>1
||Entering(2) rcount([2, [3, 4, [5, 6, 7], 8], 9])
|||Entering(3) rcount(2)
|||Exiting(3) rcount(2)=>1
|||Entering(3) rcount([[3, 4, [5, 6, 7], 8], 9])
||||Entering(4) rcount([3, 4, [5, 6, 7], 8])
|||||Entering(5) rcount(3)
|||||Exiting(5) rcount(3)=>1
|||||Entering(5) rcount([4, [5, 6, 7], 8])
||||||Entering(6) rcount(4)
||||||Exiting(6) rcount(4)=>1
||||||Entering(6) rcount([[5, 6, 7], 8])
|||||||Entering(7) rcount([5, 6, 7])
||||||||Entering(8) rcount(5)
||||||||Exiting(8) rcount(5)=>1
||||||||Entering(8) rcount([6, 7])
|||||||||Entering(9) rcount(6)
|||||||||Exiting(9) rcount(6)=>1
|||||||||Entering(9) rcount([7])
||||||||||Entering(10) rcount(7)
||||||||||Exiting(10) rcount(7)=>1
|||||||||Exiting(9) rcount([7])=>1
||||||||Exiting(8) rcount([6, 7])=>2
|||||||Exiting(7) rcount([5, 6, 7])=>3
|||||||Entering(7) rcount([8])
||||||||Entering(8) rcount(8)
||||||||Exiting(8) rcount(8)=>1

9

# functools module
- has some decorators
- [doc](https://docs.python.org/3.5/library/functools.html)


In [15]:
# here only need to define 
# __eq__ and __lt__
# the decorator defines __le__, __ge__, __le__ 

from functools import total_ordering

@total_ordering
class Student:
    def __init__(self, first, last):
        self.first = first
        self.last = last
        self.pair = (last, first)
    def __eq__(self, other):
        # instead of checking first and last names 
        # separately, make tuples 
        # and check those once
        s = (self.last.lower(), self.first.lower())
        o = (other.last.lower(), other.first.lower())
        return s == o
    def __lt__(self, other):
        s = (self.last.lower(), self.first.lower())
        o = (other.last.lower(), other.first.lower())        
        return s < o

In [16]:
# here only need to define 
# __eq__ and __lt__
# the decorator defines __le__, __ge__, __le__ 

from functools import total_ordering

@total_ordering
class Student:
    def __init__(self, first, last):
        self.first = first
        self.last = last
        # for comparison convenience
        self.pair = (last, first)
        
    def __eq__(self, other):
        return self.pair == other.pair
    
    def __lt__(self, other):
        return self.pair < other.pair

In [17]:
s1 = Student('jack', 'stead')
s2 = Student('larry', 'stead')

# only the first two operators 
# were explicitly defined above
[s1 == s2, s1 < s2, s1 > s2, s1 <= s2, s1 >= s2]

[False, True, False, True, False]

# dynamic programming/memoization
- avoid redoing computations by cacheing results

In [18]:
# f[n] = f[n-1] + f[n-2]
# doubly recursive
# many redundant calls...

def fibonacci(n):
   "Return the nth fibonacci number."
   print('in fib', n)
   if n in (0,1):
      return n
   return fibonacci(n-1) + fibonacci(n-2)

fibonacci(7)

in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1


13

In [18]:
import collections
import functools

class memoized(object):
   '''Decorator. Caches a function's return 
   value each time it is called.
   If called later with the same arguments, 
   the cached value is returned
   (not reevaluated).
   '''
   def __init__(self, func):
      self.func = func
      self.cache = {}
        
   def __call__(self, *args):
      if args in self.cache:
         # found previous computation in cache
         return self.cache[args]
      else:
         # add this computation to cache
         value = self.func(*args)
         self.cache[args] = value
         return value
    
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.func.__doc__

@memoized
def fibonaccim(n):
   "Return the nth fibonacci number."
   print('in fib', n)
   if n in (0, 1):
      return n
   return fibonaccim(n-1) + fibonaccim(n-2)

In [19]:
# now no redundant calls

fibonaccim(8)

in fib 8
in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0


21

In [21]:
# functools has a better memo decorator

import functools

# maxsize=an int will limit the size of the cache

@functools.lru_cache(maxsize=None)
def fiblru(n):
   "Return the nth fibonacci number."
   print('in fib', n)
   if n in (0, 1):
      return n
   return fiblru(n-1) + fiblru(n-2)


In [22]:
fiblru(8)

in fib 8
in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0


21

In [23]:
# info about the cache

fiblru.cache_info()

CacheInfo(hits=6, misses=9, maxsize=None, currsize=9)

In [24]:
# can clear the cache

fiblru.cache_clear()

In [25]:
fiblru.cache_info()

CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

In [26]:
# oops - can't use a list as a dict key!

@functools.lru_cache(maxsize=None)
def cnt(lst):
    return len(lst)

cnt([3,3,4])

TypeError: unhashable type: 'list'

# [Standard Library of Decorators](https://wiki.python.org/moin/PythonDecoratorLibrary)
- some useful things